In [1]:
import torch

from data.data_load import get_neckline_df, get_device, download_images
from multi_modal.data_set import get_dataset, print_dataset_names
from torch.utils.data import DataLoader
from multi_modal.fashion_clip.pretrained_model import get_model, train_step
from tqdm.auto import tqdm

In [2]:
device = get_device()
device

device(type='mps')

In [3]:
data_size = 20

train_df, test_df = get_neckline_df('neck_line_concated.csv', 'neck_line_test_item_no_list.txt', data_size=data_size)

['라운드넥' '반집업 카라' '보트넥' '브이넥' '스퀘어' '슬릿' '오프숄더' '오픈카라' '일반카라' '터틀넥/폴라'
 '헨리/차이나카라' '후드']


In [4]:
train_df

,item_no,item_name,detail_image_url_1,neck_line_label,neck_line_label_desc
1,2574285,라운드 돌먼 티셔츠 SA5STG1,https://img.29cm.co.kr/item/202404/11eef30f3af...,0,A neckline that forms a complete circle or sem...
5,2574019,뷔스티에 레이어드 니트 SA4LKH1,https://img.29cm.co.kr/item/202404/11eef307ede...,0,A neckline that forms a complete circle or sem...
6,2573900,U 핑크 디스트레스드 모헤어 점퍼 핑크 DISTRESSED MOHAIR JUMPER,https://img.29cm.co.kr/next-product/2024/04/05...,0,A neckline that forms a complete circle or sem...
7,2573898,U 모헤어 스트라이프 점퍼 버건디 옐로우 MOHAIR STRIPED JUMPER,https://img.29cm.co.kr/next-product/2024/04/05...,0,A neckline that forms a complete circle or sem...
9,2573895,U 크루넥 스웨트셔츠 베이비 블루 CREWNECK SWEATSHIRT BABY BLUE,https://img.29cm.co.kr/next-product/2024/04/05...,0,A neckline that forms a complete circle or sem...
...,...,...,...,...,...
374,2561848,WM 남성 SHADE 익스퍼트 후드티 8GTBX24105OWH,https://img.29cm.co.kr/item/202403/11eeed94719...,11,A fabric covering attached to the neckline of ...
375,2561845,WM 남성 SHADE 익스퍼트 후드티 8GTBX24105BLK,https://img.29cm.co.kr/item/202403/11eeed94552...,11,A fabric covering attached to the neckline of ...
497,2560110,NYLON PUFF SLEEVE HOODIE CHARCOAL,https://img.29cm.co.kr/next-product/2024/03/28...,11,A fabric covering attached to the neckline of ...
498,2560109,NYLON PUFF SLEEVE HOODIE PINK,https://img.29cm.co.kr/next-product/2024/03/28...,11,A fabric covering attached to the neckline of ...


In [5]:
test_df

,item_no,item_name,detail_image_url_1,neck_line_label,neck_line_label_desc
37912,1756685,Sonic Sweat Ivory,https://img.29cm.co.kr/next-product/2022/10/05...,0,A neckline that forms a complete circle or sem...
37913,1604241,MINERAL V Neck Sleeveless Lemon,https://img.29cm.co.kr/next-product/2022/06/21...,3,"A neckline that dips down into a V-shape, typi..."
37914,1729581,WOOL OVERSIZED SWEATER BLUE,https://img.29cm.co.kr/next-product/2022/09/20...,2,"A wide, horizontal neckline that sits near the..."
37915,1361806,UNISEX 스몰 레오파드 자수 스마일 맨투맨 5color,https://img.29cm.co.kr/next-product/2021/12/29...,0,A neckline that forms a complete circle or sem...
37916,955946,여성 아머럭스 레스코닐 BI8 네이비X화이트,https://img.29cm.co.kr/next-product/2023/08/21...,0,A neckline that forms a complete circle or sem...
...,...,...,...,...,...
38649,1989574,W 소프트 슬릿 가디건 라이트그린,https://img.29cm.co.kr/next-product/2023/04/03...,5,"A narrow opening or cut in the neckline, sleev..."
38650,1438567,ROUND NECK SHIRTS T BLACK,https://img.29cm.co.kr/next-product/2022/03/04...,5,"A narrow opening or cut in the neckline, sleev..."
38651,1438609,ROUND NECK SHIRTS T CREAM,https://img.29cm.co.kr/next-product/2022/03/04...,5,"A narrow opening or cut in the neckline, sleev..."
38652,1982571,트임넥 변형 싱글 스판 티셔츠 베이지,https://img.29cm.co.kr/next-product/2023/03/21...,5,"A narrow opening or cut in the neckline, sleev..."


In [6]:
train_urls = [url for url in train_df['detail_image_url_1'].tolist()]
train_labels = train_df['neck_line_label'].tolist()
train_item_no_list = train_df['item_no'].tolist()
train_sentences = train_df['neck_line_label_desc'].tolist()

test_urls = [url for url in test_df['detail_image_url_1'].tolist()]
test_labels = test_df['neck_line_label'].tolist()
test_item_no_list = test_df['item_no'].tolist()
test_sentences = test_df['neck_line_label_desc'].tolist()

In [7]:
download_images(train_urls)
download_images(test_urls)

  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

In [8]:
batch_size = 64

print_dataset_names()
dataset = get_dataset(train_item_no_list, train_urls, train_sentences, train_labels, 'CustomDatasetWithOutProcessor')
train_dataloader = DataLoader(dataset, batch_size=batch_size)

CustomDatasetWithProcessor CustomDatasetWithOutProcessor CustomDatasetWithPreprocessor


In [9]:
clip_model = get_model(device)

In [10]:
lr=1e-6

optimizer = torch.optim.Adam(clip_model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-6, weight_decay=0.2)

img_criterion = torch.nn.CrossEntropyLoss()
txt_criterion = torch.nn.CrossEntropyLoss()

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=2, eta_min=0)

In [ ]:
num_epochs = 1000

for epoch in range(num_epochs):
    clip_model.train()
    running_loss = 0.0
    running_img_loss = 0.0
    running_txt_loss = 0.0
    predictions = []
    targets = []

    # for batch_data in tqdm(train_dataloader):
    for batch_data in train_dataloader:
        optimizer.zero_grad()
        
        outputs = train_step(clip_model, device, batch_data, return_loss=True)

        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        target = torch.arange(len(logits_per_image), device=device)
        
        # img_loss = torch.nn.functional.cross_entropy(logits_per_image, target)
        img_loss = img_criterion(logits_per_image, target)
        # txt_loss = torch.nn.functional.cross_entropy(logits_per_text, target)
        txt_loss = txt_criterion(logits_per_text, target)
        
        loss = (img_loss + txt_loss) / 2

        # loss = (txt_criterion(logits_per_text, batch_labels) + img_criterion(logits_per_image, batch_labels)) / 2

        # loss = txt_criterion(logits_per_text, batch_labels)

        # loss = img_criterion(logits_per_text, batch_labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_img_loss += img_loss.item()
        running_txt_loss += txt_loss.item()
        
    scheduler.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader)}, Img Loss: {running_img_loss / len(train_dataloader)}, Txt Loss: {running_txt_loss / len(train_dataloader)}, LR: {scheduler.get_last_lr()}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
